<a href="https://colab.research.google.com/github/shitote/dist-repo/blob/main/introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NLP are also refered to as a sequence to sequence problems.(seq2seq)

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# import helper functions.
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-05-03 02:00:36--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-03 02:00:37 (53.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
Kaggle's introduction to NLP dataset (text samples of Tweets labbelled as disaster or not disarster)

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-05-03 02:00:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.27, 172.217.7.59, 172.217.12.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2024-05-03 02:00:45 (50.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset.

Read the text file in the notebook

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# Note that the test data frame have no target values.
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# Find Out if the dataset is baklanced or not?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# Find the number of total samples.
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# visualize random training examples.
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+10].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(Not real disaster)")
  print(f"Text:\n{text}\n")
  print("-----\n")

Target: 1 (real disaster)
Text:
Abe's government made clear that grenades were not 'weapon.'

-----

Target: 0 (Not real disaster)
Text:
'By replacing fear of the unknown with curiosity we open ourselves up toÛ_ https://t.co/qRHng6kJ1C

-----

Target: 1 (real disaster)
Text:
@nataliealund 
Parents of Colorado theater shooting victim fear copycat massacre

http://t.co/LvlH3W3aWO
#Antioch

http://t.co/vIwXY1XDYK

-----

Target: 1 (real disaster)
Text:
Earthquake and tsunami that occurred in Japan 'free speech' is also swallowed. http://t.co/TJyyFT6NV0

-----

Target: 0 (Not real disaster)
Text:
I can't listen to Darude Sandstorm without expecting airhorns now

-----

Target: 1 (real disaster)
Text:
#Nuclear policy of #Japan without responsibility about Nuclear #Disaster will repeat same #failure.
#annonymous #guardian #NYTimes #Reuters

-----

Target: 1 (real disaster)
Text:
Warfighting Robots Could Reduce Civilian Casualties So Calling for a Ban Now Is Premature http://t.co/lzff4pT4AZ 

### Split the data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Coverting text to numbers
* Tokenization - This is direct mapping of token to number
* Emmbedding - Create a matrix of feature vectors for each token (
    the size fo the feature vector can be defined for each token embedding can be learned
)

### Text vectorization (tokenization)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters.
text_vectorizer = TextVectorization(max_tokens=None,  # How many words in the vocabulary (automaticaly add <oov>)
                                    standardize='lower_and_strip_punctuation',  # Remove the fluf
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False
                                    )

In [15]:
len(train_sentences[0].split())

7

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# set text vectorization variables
max_vocab_length = 1000
max_length = 16

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the trainin text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There are floods everywhere"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 16), dtype=int64, numpy=
array([[ 74,  22, 207,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])>

In [20]:
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence}\n\n Vectorized version:')
text_vectorizer([random_sentence])

Original text: 
 @anarchic_teapot @BoironUSA @zeno001 Glononium 6C also helps with being blown up while bashing a bottle of nitroglycerin against a book.

 Vectorized version:


<tf.Tensor: shape=(1, 16), dtype=int64, numpy=
array([[  1,   1,   1,   1,   1, 336,   1,  14, 121, 468,  27, 211,   1,
          3,   1,   6]])>

In [21]:
# et the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words i vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words i vocab: 1000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['reported', 'r', 'pray', 'playlist', 'patience']


In [22]:
### Create an Embedding using an embedding layer
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,  # set the input shape
                            output_dim=128,  # output shape
                            input_length=max_length # how long is each input
                            )

embedding


In [23]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence} \
      \n\nEmbedding version: ")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over southern China by SÛ_ http://t.co/0XCb7yeqmw       

Embedding version: 


<tf.Tensor: shape=(1, 16, 128), dtype=float32, numpy=
array([[[ 0.00234103, -0.03596921, -0.01634117, ..., -0.03000554,
          0.02053742, -0.01262529],
        [-0.04071338,  0.01796693,  0.03035528, ...,  0.04684467,
         -0.00928558,  0.01547224],
        [ 0.00234103, -0.03596921, -0.01634117, ..., -0.03000554,
          0.02053742, -0.01262529],
        ...,
        [-0.00316513,  0.0374526 ,  0.02494517, ...,  0.03852928,
          0.00043613,  0.02588094],
        [ 0.00234103, -0.03596921, -0.01634117, ..., -0.03000554,
          0.02053742, -0.01262529],
        [ 0.00901936,  0.03853954, -0.03249051, ..., -0.02094114,
         -0.01407913,  0.00011941]]], dtype=float32)>

## Modelling a text dataset (running a serries of experiments)

The data is now in numbers it is time to build a series of modelling experiments.

* model 0: naive bayes (baseline), this is from sklearn ML
* model 1: Feed-forward neural network (dence model)
* model 2: LSTM model (RNN)
* model 3: GRU model (RNN)
* model 4: Bidirectional Neural Networks(RNN)
* model 5: !D Convolutional neural nerworks (CNN)
* model 6: Tensoflow hub Pretrained Feature extraction (using transfer learnin NLP)
* model 7: Same as model 6 with 10% of training data.

THs use od standard steps of modelling in tensorflow:
* Crreate a model.
* Build a model.
* FIt the model.
* Evaluate the model.

### Model 0: Getting a baseline.

**NOte** It is a better proctice to use non-DL alorithms to create the baseline model since they are fast and easy to train then latter impliment it with the DL to improve the performance.

In [24]:
# Trun the data into numerical.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create a tokenization and modellin pipeline.
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),   #Convert words to numbers.
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
# Baseline model evaluation.
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline model score: {baseline_score*100:.2f}%")

Baseline model score: 79.27%


In [26]:
# Make predictions.
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

In [28]:
# Fuction to evalate: accuracy, precision, recall, f1_score.
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results


In [29]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model_1: A simple dence model

In [30]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

SAVE_DIR = 'model_logs'

In [31]:
# Build a model using the functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)  # Input are 1-dimensional array of string.
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 16)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 16, 128)           128000    
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 128129 (500.50 KB)
Trainable params: 12

In [33]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [34]:
train_sentences.shape, train_labels.shape

((6851,), (6851,))

In [35]:
train_labels = tf.reshape(train_labels, (-1, 1))
val_labels = tf.reshape(val_labels, (-1, 1))

In [36]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name='model_1_dence')])

Saving TensorBoard log files to: model_logs/model_1_dence/20240503-020047
Epoch 1/5
215/215 [==============================] - 2s 6ms/step - loss: 0.6201 - accuracy: 0.6650 - val_loss: 0.5601 - val_accuracy: 0.7362
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4925 - accuracy: 0.7812 - val_loss: 0.4988 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4380 - accuracy: 0.8078 - val_loss: 0.4847 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4109 - accuracy: 0.8178 - val_loss: 0.4791 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3938 - accuracy: 0.8247 - val_loss: 0.4820 - val_accuracy: 0.7835


In [37]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 10ms/step - loss: 0.4820 - accuracy: 0.7835


[0.4820061922073364, 0.7834645509719849]

In [38]:
# Make somce Predictions and evaluate them
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 1s 8ms/step


(762, 1)

In [39]:
model_1_pred_probs[0]

array([0.5265502], dtype=float32)

In [40]:
# Convert the models prediction into probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [41]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.34645669291339,
 'precision': 0.7880043492637875,
 'recall': 0.7834645669291339,
 'f1': 0.7804929895574078}

In [42]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualize the learned embedding.

In [43]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(1000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 16)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 16, 128)           128000    
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 128129 (500.50 KB)
Trainable params: 12

In [45]:
# Get the wight metrix of embeddin layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(1000, 128)

### Create embedding files.

In [46]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + '\n')
  out_m.write(word + '\n')
out_v.close()
out_m.close()

In [47]:
# Dowload file from colab to upload to the projector
# word to vector jay alama
try:
  from google.colab import files
  files.download("vectors.tsv")
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN)

RNN are useful for sequence data.

The premise of a recurrent neuaral network is to use the representatoin of a previous input to aid the representation of a later input

The internals of RNN:
- MIT's sequence modelling.
- Chris olah's intor to LSTMs:
- Andrej Kapathy's the unreasonable effectiveness of recurrent neural networks.

### Model 2: LSTM
LSTM = Long short term memory (One of the most popular LSTM cells)
* The structure of an RNN trypically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probabilities)

In [48]:
# Tenh
# Create an LSTM model.
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [49]:
# Compile the model.
model_2.compile(loss='binary_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=["accuracy"])

In [50]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240503-020112
Epoch 1/5
215/215 [==============================] - 17s 30ms/step - loss: 0.4167 - accuracy: 0.8159 - val_loss: 0.5186 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3800 - accuracy: 0.8327 - val_loss: 0.5100 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3719 - accuracy: 0.8389 - val_loss: 0.5029 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.3621 - accuracy: 0.8445 - val_loss: 0.4863 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 6s 30ms/step - loss: 0.3451 - accuracy: 0.8518 - val_loss: 0.5185 - val_accuracy: 0.7743


In [51]:
# Make predictions with the LSTM model.
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[0.68543226],
       [0.6314952 ],
       [0.89599353],
       [0.06244384],
       [0.04637371],
       [0.9626955 ],
       [0.8919058 ],
       [0.7996221 ],
       [0.6049916 ],
       [0.15919301]], dtype=float32)

In [52]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [53]:
# Calculate model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy': 77.42782152230971,
 'precision': 0.7759894665484696,
 'recall': 0.7742782152230971,
 'f1': 0.7722311836526509}

### Model 3: GRU

This is also a popular RNN component GRU - gated recurrent Unit

The GRU have the same features to an LSTM cell but have less parameters,

RFewources:
understantin the GRU networks by simion


In [54]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
# x = layers.LSTM(64)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [55]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 16)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 16, 128)           128000    
                                                                 
 gru (GRU)                   (None, 16, 64)            37248     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                       

In [56]:
# Compile the model
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [57]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240503-020204
Epoch 1/5
215/215 [==============================] - 12s 33ms/step - loss: 0.3822 - accuracy: 0.8330 - val_loss: 0.5098 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.3372 - accuracy: 0.8624 - val_loss: 0.5492 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 6s 30ms/step - loss: 0.3174 - accuracy: 0.8672 - val_loss: 0.5268 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2922 - accuracy: 0.8772 - val_loss: 0.6044 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.2727 - accuracy: 0.8853 - val_loss: 0.6281 - val_accuracy: 0.7638


In [58]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 12ms/step


array([[0.7744783 ],
       [0.5412183 ],
       [0.980894  ],
       [0.03348786],
       [0.04822764],
       [0.9983059 ],
       [0.9983175 ],
       [0.9152347 ],
       [0.45253804],
       [0.14884639]], dtype=float32)

In [59]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 0., 0.], dtype=float32)>

In [60]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.37795275590551,
 'precision': 0.7684672893281957,
 'recall': 0.7637795275590551,
 'f1': 0.7601256218811575}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (the same way you will read a kiswahili sentence).
The bidirection RNN go from right to let as well as from  left to right

In [61]:
# Build a bidirection RNN
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [62]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 16)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 16, 128)           128000    
                                                                 
 bidirectional (Bidirection  (None, 16, 128)           98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                             

In [63]:
model_4.compile(loss="binary_crossentropy",
                optimizer= tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [64]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240503-020253
Epoch 1/5
215/215 [==============================] - 21s 61ms/step - loss: 0.3418 - accuracy: 0.8599 - val_loss: 0.6576 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 10s 45ms/step - loss: 0.2836 - accuracy: 0.8819 - val_loss: 0.5926 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 10s 47ms/step - loss: 0.2537 - accuracy: 0.8926 - val_loss: 0.6614 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 10s 47ms/step - loss: 0.2360 - accuracy: 0.9031 - val_loss: 0.8056 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 10s 45ms/step - loss: 0.2137 - accuracy: 0.9102 - val_loss: 0.8788 - val_accuracy: 0.7520


In [65]:
# make predictions.
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 2s 11ms/step


array([[0.91961133],
       [0.7369214 ],
       [0.9990952 ],
       [0.00637378],
       [0.01271807],
       [0.9997022 ],
       [0.9982266 ],
       [0.90455705],
       [0.49197474],
       [0.10514459]], dtype=float32)

In [66]:
# Turn the results into a comparable features.
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 0., 0.], dtype=float32)>

In [67]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 75.19685039370079,
 'precision': 0.7555317501896176,
 'recall': 0.7519685039370079,
 'f1': 0.7483944946347455}

## Convolutional Neural Networks for text and othet types od seu=quences

The typical structure of a convolutional neural network

Input (text) -> Tokenization -> Embedding -> Layers (typically conv1D + pooling) -> Outputs (class probabilities)

### Model 5: conv1D

In [68]:
# Make a test of the embedding layer, conv1D layer and max pooling layers
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test)  # pass the test embeddin into the conv1d layer.
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)   # get the most important fearutes.

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 16, 128]), TensorShape([1, 12, 32]), TensorShape([1, 32]))

In [69]:
embedding_test

<tf.Tensor: shape=(1, 16, 128), dtype=float32, numpy=
array([[[-0.04646016, -0.0227955 ,  0.00952806, ..., -0.03665891,
          0.03394498, -0.03475453],
        [-0.06952136,  0.00077504, -0.00705832, ..., -0.00152895,
          0.06788455, -0.01484976],
        [-0.00734625, -0.00472314, -0.01666687, ...,  0.0749933 ,
         -0.01066959,  0.04630578],
        ...,
        [-0.00185353, -0.0119912 , -0.03794016, ..., -0.01512582,
          0.01430289, -0.03709945],
        [-0.00185353, -0.0119912 , -0.03794016, ..., -0.01512582,
          0.01430289, -0.03709945],
        [-0.00185353, -0.0119912 , -0.03794016, ..., -0.01512582,
          0.01430289, -0.03709945]]], dtype=float32)>

In [70]:
conv_1d_output[:2]

<tf.Tensor: shape=(1, 12, 32), dtype=float32, numpy=
array([[[0.        , 0.00029082, 0.        , 0.01883054, 0.0207816 ,
         0.        , 0.07791423, 0.07746175, 0.03478591, 0.        ,
         0.04912568, 0.01897906, 0.0127512 , 0.03107884, 0.04219447,
         0.00164449, 0.01033248, 0.        , 0.        , 0.        ,
         0.03703111, 0.        , 0.01490187, 0.08722756, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.00207462, 0.        ],
        [0.        , 0.00546439, 0.02549405, 0.05965556, 0.        ,
         0.02380529, 0.03474809, 0.01798245, 0.03448235, 0.02019503,
         0.07709082, 0.07873905, 0.01513248, 0.        , 0.        ,
         0.        , 0.03822038, 0.0068188 , 0.05244359, 0.08131886,
         0.        , 0.        , 0.06841002, 0.00901655, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.00366383, 0.        ],
        [0.        , 0.01456065, 0.0006885 , 0.    

In [71]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.01255438, 0.01456065, 0.02549405, 0.05965556, 0.0207816 ,
        0.04889496, 0.07791423, 0.07746175, 0.03478591, 0.02019503,
        0.07709082, 0.07873905, 0.01584829, 0.03734714, 0.04219447,
        0.0332034 , 0.03822038, 0.0068188 , 0.05244359, 0.08131886,
        0.03703111, 0.01360436, 0.06841002, 0.08722756, 0.00812226,
        0.        , 0.02947932, 0.02268484, 0.01266141, 0.        ,
        0.00366383, 0.00438599]], dtype=float32)>

In [72]:
# Create d dimensional convolutional neural network layer to model sequences.
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu', padding= 'valid')(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name='model_5_conv1D')

# Compile the model and fit it ata one go
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_5.summary()

Model: "model_5_conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 16)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 16, 128)           128000    
                                                                 
 conv1d_1 (Conv1D)           (None, 12, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 1)              

In [73]:
train_sentences_tensor = tf.convert_to_tensor(train_sentences)
val_sentences_tensor = tf.convert_to_tensor(val_sentences)

In [74]:
# fit model 5
model_5_history = model_5.fit(train_sentences_tensor,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences_tensor, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'conv1D')])


Saving TensorBoard log files to: model_logs/conv1D/20240503-020425
Epoch 1/5
215/215 [==============================] - 3s 9ms/step - loss: 0.3720 - accuracy: 0.8375 - val_loss: 0.5547 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3329 - accuracy: 0.8572 - val_loss: 0.5826 - val_accuracy: 0.7559
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3074 - accuracy: 0.8699 - val_loss: 0.6041 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2761 - accuracy: 0.8869 - val_loss: 0.6054 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2433 - accuracy: 0.9038 - val_loss: 0.6272 - val_accuracy: 0.7546


 ## # Model 6: Tensorflow Hub Pretrained Sentence Encoder

In [75]:
# import tensorflow_hub as hub
# import tensorflow as tf

# english_sentences = tf.constant(["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."])

# preprocessor = hub.KerasLayer(
#     "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
# encoder = hub.KerasLayer(
#     "https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/cmlm-en-base/1")

# english_embeds = encoder(preprocessor(english_sentences))["default"]

# print (english_embeds)


In [76]:
# !pip install tensorflow-text

In [77]:
# !pip install --upgrade tensorflow

In [78]:
import tensorflow as tf

print(tf.__version__)

2.15.0


from json import encoder
## Model 6: Tensorflow hub pretrained sentence encoder
Transfer learning for NLP, Specificaly use tensorflow hub universal sentence encoder

In [83]:
import tensorflow_hub as hub

In [85]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
embed = model
embed_samples = embed([sample_sentence,
                       "When you add the universal sentences encoder to a sentence, it converts it to numbers"])
print(embed_samples[0][:50])


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded
tf.Tensor(
[-0.05953072  0.009958    0.05262217  0.03299494  0.03481619  0.03550496
  0.03030668  0.05389166 -0.04455226  0.01692487  0.0261747  -0.00416461
  0.02361121  0.06394361  0.05907685 -0.03532546  0.03982759 -0.02722584
 -0.01726626 -0.0558493  -0.02147491  0.07448269  0.00204131  0.0272521
 -0.00876293 -0.05909175  0.01170975 -0.00945128  0.01568378  0.05079331
 -0.01973779 -0.02515237 -0.02078041 -0.01900205  0.00385312 -0.02912632
  0.03036552  0.06737051  0.00844657 -0.05294723 -0.06610619  0.00431143
 -0.0285227   0.05258462 -0.09454233  0.00011859 -0.0305081   0.00793438
 -0.05065931  0.03487634], shape=(50,), dtype=float32)


In [86]:
embed_samples

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.05953072,  0.009958  ,  0.05262217, ...,  0.01666443,
         0.00035226, -0.03008756],
       [ 0.05224953, -0.0920862 , -0.01916212, ..., -0.04403271,
         0.03968364,  0.00878738]], dtype=float32)>

In [87]:
# Create a keras layer using pretrained layer from Hub
sentence_encoder_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4",
    input_shape=[],
    dtype=tf.string,
    trainable=False,
    name="USE"
    )

In [90]:
# Create the model using sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
], name='model_6_USE')

In [91]:
# Compile the model
model_6.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [92]:
# TRain a classifier on top of USE embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'tf_hub_sentence_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20240503-025221
Epoch 1/5
215/215 [==============================] - 10s 17ms/step - loss: 0.5058 - accuracy: 0.7759 - val_loss: 0.4505 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4133 - accuracy: 0.8161 - val_loss: 0.4408 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4008 - accuracy: 0.8221 - val_loss: 0.4353 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.3927 - accuracy: 0.8253 - val_loss: 0.4312 - val_accuracy: 0.8123
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3851 - accuracy: 0.8298 - val_loss: 0.4280 - val_accuracy: 0.8150


In [93]:
# make predictions with USE TF HUb model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 2s 25ms/step


array([[0.17552035],
       [0.8020521 ],
       [0.98686177],
       [0.21336213],
       [0.7589956 ],
       [0.7431257 ],
       [0.98104   ],
       [0.98251873],
       [0.9446892 ],
       [0.10250424]], dtype=float32)

In [95]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [96]:
# Calculate the model 6 performance metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8159654539826816,
 'recall': 0.8149606299212598,
 'f1': 0.8139016283708382}

In [97]:
# improve the preformance of the  model
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
], name='model_7_USE')

In [99]:
# Compile the model.
model_7.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_11 (Dense)            (None, 64)                32832     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256834881 (979.75 MB)
Trainable params: 37057 (144.75 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [108]:
# Fit the model.
model_7_history = model_7.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_7_USE')])

Saving TensorBoard log files to: model_logs/model_7_USE/20240503-031829
Epoch 1/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0863 - accuracy: 0.9714 - val_loss: 0.7718 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0741 - accuracy: 0.9777 - val_loss: 0.7789 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0691 - accuracy: 0.9800 - val_loss: 0.9178 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0642 - accuracy: 0.9800 - val_loss: 0.9086 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0570 - accuracy: 0.9834 - val_loss: 0.9051 - val_accuracy: 0.7966


In [109]:
# Make predictions for the model.
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:5]

24/24 [==============================] - 0s 20ms/step


array([[6.9130363e-04],
       [9.9563384e-01],
       [1.0000000e+00],
       [4.5415591e-03],
       [1.6593790e-01]], dtype=float32)

In [110]:
# make the prediction result comparerable.
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [111]:
# calculate model 7 presormance metrics.
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

{'accuracy': 79.65879265091863,
 'precision': 0.7971332811059624,
 'recall': 0.7965879265091863,
 'f1': 0.795517158602831}

## Model 7: TF HUB Pretrained USE with only 10% of the training data

In [116]:
# create subsets of 10% of training data
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_10_percent.head(), len(train_10_percent), len(train_df_shuffled)

(                                                   text  target
 4955  DFR EP016 Monthly Meltdown - On Dnbheaven 2015...       0
 584   FedEx no longer to transport bioterror germs i...       0
 7411  Gunmen kill four in El Salvador bus attack: Su...       1
 5950  @camilacabello97 Internally and externally scr...       1
 5541  Radiation emergency #preparedness starts with ...       1,
 761,
 7613)

In [117]:
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [120]:
# Check the number of target in subset of data
train_10_percent['target'].value_counts()

target
0    413
1    348
Name: count, dtype: int64

In [123]:
train_sentences_10_percent[:5]

['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
 'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
 'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
 '@camilacabello97 Internally and externally screaming',
 'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov']

#### Clone model 6 to adapt it to model 7.

In [125]:
model_7 = tf.keras.models.clone_model(model_6)

# compile the model.
model_7.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________
